In [1]:
%reload_ext autoreload
%autoreload 2

The history saving thread hit an unexpected error (DatabaseError('database disk image is malformed',)).History will not be written to the database.


In [2]:
import copy
from pprint import pprint
from pathlib import Path
from typing import Dict, List
import spacy
import srsly
# import recon
from recon.corpus import Corpus
from recon.constants import NONE
from recon.corrections import fix_annotations
from recon.dataset import Dataset
from recon.loaders import read_jsonl
from recon.types import Example, PredictionError, HardestExample, NERStats, EntityCoverageStats, EntityCoverage
from recon.stats import (
    get_ner_stats, get_entity_coverage, get_sorted_type_counts, counts_to_probs, entropy,
    calculate_entity_coverage_entropy, calculate_label_balance_entropy, calculate_label_distribution_similarity,
    detect_outliers
)
import recon.tokenization as tokenization
from recon.insights import get_ents_by_label, get_label_disparities, top_prediction_errors, top_label_disparities, get_hardest_examples
from recon.recognizer import SpacyEntityRecognizer
from recon.operations import registry
from recon.store import ExampleStore

In [3]:
registry.operations.get_all()

OrderedDict([('fix_tokenization_and_spacing',
              <function recon.operations.operation.__call__.<locals>.factory(data, **cfg) -> recon.types.OperationResult>),
             ('add_tokens',
              <function recon.operations.operation.__call__.<locals>.factory(data, **cfg) -> recon.types.OperationResult>)])

In [4]:
fix_tokenization_and_spacing = registry.operations.get("fix_tokenization_and_spacing")
add_tokens = registry.operations.get("add_tokens")

In [5]:
corpus = Corpus.from_disk("./data/skills")

In [17]:
corpus.apply_(fix_tokenization_and_spacing)

Found 3 tokenization errors.
Found 1 unfixable tokenization errors.
[('We are looking for a Software Development Engineer who has solid coding skills, a strong machine learning background, and is passionate about developing new AI products.', 'Software Development Engineer'), ('We are looking for a Software Development Engineer who has solid coding skills, a strong machine learning background, and is passionate about developing new AI products.', 'coding'), ('Responsibilities As a SOFTWARE DEVELOPMENT ENGINEER II you will work / collaborate with other talented engineers to build features and technologies that will affect millions of your fellow developers in the community.', 'SOFTWARE DEVELOPMENT ENGINEER ')]
{545202776435063599} set() 0 1 105 106
set() set() 0 0 110 110
set() set() 0 0 96 96


In [18]:
corpus.apply_(add_tokens)

set() set() 0 0 105 105
set() set() 0 0 110 110
set() set() 0 0 96 96


In [7]:
large_corpus = Corpus.from_disk("../../CognitiveServices/API-TextAnalytics-NER.CloudServices/data/2020-03-30")

In [41]:
large_corpus.example_store, large_corpus._train.example_store

(<recon.store.ExampleStore at 0x7fe35d856f98>,
 <recon.store.ExampleStore at 0x7fe35d856f98>)

In [32]:
%timeit hash(corpus._train), hash(corpus._dev), hash(corpus._test)

2.85 ms ± 199 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [33]:
%timeit hash(large_corpus._train), hash(large_corpus._dev), hash(large_corpus._test)

75.1 ms ± 8.1 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [34]:
%timeit hash(corpus._train)
%timeit hash(large_corpus._train)

872 µs ± 69.8 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
59.9 ms ± 8.17 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [36]:
len(str(hash(corpus._train.data[0])))

18

In [28]:
hash(corpus._train.data[0].spans[0])

2042131441891384299

In [12]:
corpus._train.commit_hash

'0dd8ebeed75a419cbb1f7e0a9b6244a46e2eebfd'

In [13]:
print(fix_tokenization_and_spacing)

<function operation.__call__.<locals>.factory at 0x7f3f1df8eb70>


In [14]:
corpus._train.operations = []
corpus._train.global_state = {}
corpus._dev.operations = []
corpus._dev.global_state = {}
corpus._test.operations = []
corpus._test.global_state = {}

In [9]:
corpus.apply_(fix_tokenization_and_spacing)

Found 1 tokenization errors.
Found 0 unfixable tokenization errors.
{860580076040132479}
set()
set()


In [20]:
corpus._train.operations

[OperationState(name='fix_tokenization_and_spacing', status=<OperationStatus.COMPLETED: 'COMPLETED'>, ts=datetime.datetime(2020, 4, 7, 13, 4, 42, 400661), examples_added=0, examples_removed=1, examples_corrected=0, annotations_added=0, annotations_removed=0, annotations_corrected=0, transformations=[]),
 OperationState(name='fix_tokenization_and_spacing', status=<OperationStatus.COMPLETED: 'COMPLETED'>, ts=datetime.datetime(2020, 4, 7, 13, 4, 42, 400661), examples_added=0, examples_removed=0, examples_corrected=0, annotations_added=0, annotations_removed=0, annotations_corrected=0, transformations=[]),
 OperationState(name='fix_tokenization_and_spacing', status=<OperationStatus.COMPLETED: 'COMPLETED'>, ts=datetime.datetime(2020, 4, 7, 13, 4, 42, 400661), examples_added=0, examples_removed=0, examples_corrected=0, annotations_added=0, annotations_removed=0, annotations_corrected=0, transformations=[])]

In [21]:
corpus.apply_(add_tokens)

name='add_tokens' status=<OperationStatus.COMPLETED: 'COMPLETED'> ts=datetime.datetime(2020, 4, 7, 13, 4, 42, 400661) examples_added=0 examples_removed=0 examples_corrected=0 annotations_added=0 annotations_removed=0 annotations_corrected=0 transformations=[]
{'fix_tokenization_and_spacing': OperationState(name='fix_tokenization_and_spacing', status=<OperationStatus.COMPLETED: 'COMPLETED'>, ts=datetime.datetime(2020, 4, 7, 13, 4, 42, 400661), examples_added=0, examples_removed=0, examples_corrected=0, annotations_added=0, annotations_removed=0, annotations_corrected=0, transformations=[]), 'add_tokens': OperationState(name='add_tokens', status=<OperationStatus.COMPLETED: 'COMPLETED'>, ts=datetime.datetime(2020, 4, 7, 13, 4, 42, 400661), examples_added=0, examples_removed=0, examples_corrected=0, annotations_added=0, annotations_removed=0, annotations_corrected=0, transformations=[])}
name='add_tokens' status=<OperationStatus.COMPLETED: 'COMPLETED'> ts=datetime.datetime(2020, 4, 7, 13, 

In [22]:
corpus.to_disk('./fixed_data/skills', force=True)

In [141]:
import os
os.listdir('./fixed_data/skills/.recon/train')

['state.json']